In [38]:
import quantumpropagator as qp
import matplotlib.pyplot as plt
from matplotlib import cm
from ipywidgets import interact,fixed #, interactive, fixed, interact_manual
import ipywidgets as widgets
import os
import glob
import pandas as pd
#%matplotlib notebook
%matplotlib ipympl

plt.rcParams.update({'font.size': 10})

In [39]:
fol = '/home/alessio/Desktop/Dropbox/USA/bet/'
subfolders = sorted([dir for dir in os.listdir(fol) if os.path.isdir(os.path.join(fol,dir)) and dir != os.path.join(fol,'.ipynb_checkpoints')])
print(''.join(['{} -> {}\n'.format(a,b) for a,b in enumerate(subfolders)]))

0 -> b-pulseAlongX_0.22_goodG_0000
1 -> b-pulseAlongX_0.22_short_0_goodG_0000
2 -> b-pulseAlongX_0.22_short_pi_goodG_0000
3 -> c-Experiments_on_Nac_0000
4 -> d-experiments_smooth01_0000
5 -> d-experiments_smooth10_0000
6 -> d-experiments_smooth50_0000
7 -> h-IR-X_80_0000
8 -> i-IR-Z_80_0000
9 -> k-IR-Y_80_0000
10 -> l-IR-XZ-50_0000
11 -> l-IR-XZ-70_0000
12 -> l-IR-XZ-90_0000
13 -> m-only_IR_longer_with_nac_2_1_0000
14 -> m-only_IR_longer_with_nac_2_1_counterClock_0000
15 -> m-only_IR_longer_with_nac_2_1_inverted_phase_0000
16 -> m-only_IR_longer_with_nac_2_1_phase_pi_0000
17 -> m-only_IR_middle100_0000
18 -> m-only_IR_middle140_0000
19 -> m-only_IR_middle50_0000
20 -> m-only_IR_middle80_0000
21 -> m-only_IR_middle90_0000
22 -> m-only_IR_retarded75_0000
23 -> m-only_IR_retarded80_0000
24 -> m-only_IR_retarded85_0000
25 -> m-only_IR_retarded90_0000
26 -> m-only_IR_short_with_nac_2_1_0000
27 -> m-only_IR_short_with_nac_2_1_inverted_phase_0000
28 -> m-only_IR_short_with_nac_2_1_phase_pi_00

In [49]:
i_want = [0]
selected = [ subfolders[i] for i in i_want ]

In [50]:
viridis = cm.get_cmap('hsv',12)
colors = viridis(np.linspace(0,1, len(i_want)+1))

def graph_between_bonds(folder_list):
    fig, [ax0,ax1] = plt.subplots(2,1,figsize=(10,10))
    
    
    for i, this_folder in enumerate(folder_list):
        abs_path = os.path.join(fol,this_folder)
        fir_file = os.path.join(abs_path, 'list_first.dat')
        sec_file = os.path.join(abs_path, 'list_second.dat')
        population_file = os.path.join(abs_path, 'outputPopul')
        dataP = pd.read_csv(population_file, delim_whitespace=True, header=None)
        fir = np.loadtxt(fir_file)
        sec = np.loadtxt(sec_file)
        norm = 0.317460*0.317460*0.317460
        ax0.plot(fir[:,0],fir[:,1]*norm, color=colors[i], ls='-', label='{} single bond'.format(this_folder))
        ax0.plot(sec[:,0],sec[:,1]*norm, color=colors[i], ls='--', label='{} double bond'.format(this_folder))
        ax1.plot(dataP[0],dataP[2], color=colors[i], ls='-', label=r'{} $S_1$'.format(this_folder))
    
    ax1.set_ylim([0,1.01])
    ax0.legend()
    ax1.legend()
    ax0.set_title(this_folder)
    ax0.set_xlabel('fs')
    ax0.set_ylabel('Fraction of 50 electrons')
    
    fig.tight_layout()
        
graph_between_bonds(selected)

A Jupyter Widget

# Blender visualizer

In [57]:
file_geom = '/home/alessio/prova_bonds_cylinder/geoms.p'
file_fs = '/home/alessio/prova_bonds_cylinder/first_second.p'

geoms = qp.pickleLoad(file_geom)
fs = qp.pickleLoad(file_fs)


In [58]:
geoms.shape
flat_geoms = geoms.reshape(-1, *geoms.shape[-2:])
flat_geoms.shape,fs['list_first'].shape, fs['list_second'].shape

((65000, 15, 3), (65000,), (65000,))

In [59]:
xmin, ymin, zmin = -10,-10,-10
nx, ny, nz = 64,64,64
x = np.linspace(xmin,-xmin,nx)
y = np.linspace(ymin,-ymin,ny)
z = np.linspace(zmin,-zmin,nz)
dx = x[1]-x[0]
dy = y[1]-y[0]
dz = z[1]-z[0]

B,A,C = np.meshgrid(x,y,z)
list_of_points_in_3d = np.stack([A.flatten(),B.flatten(),C.flatten()]).T

In [60]:
list_of_points_in_3d.shape
list_of_points_in_3d[137496]

array([ 0.47619048,  1.42857143, -2.38095238])

In [63]:
index = 30000

geom = flat_geoms[index]
first = fs['list_first'][index]
second = fs['list_second'][index]

geom.shape, first.shape, second.shape

((15, 3), (384,), (190,))

In [64]:
qp.saveTraj(np.array([geom]),['C','C','C','H','H','H','H','C','C','C','C','H','H','H','H'],'/home/alessio/prova_bonds_cylinder/norb', True)
selected_grids = list_of_points_in_3d[first]
n_gridpoints, _ = selected_grids.shape
atomT = ['H']*n_gridpoints
qp.saveTraj(np.array([selected_grids]), atomT, '/home/alessio/prova_bonds_cylinder/first',True)


file /home/alessio/prova_bonds_cylinder/norb.xyz written:

vmd /home/alessio/prova_bonds_cylinder/norb.xyz

file /home/alessio/prova_bonds_cylinder/first.xyz written:

vmd /home/alessio/prova_bonds_cylinder/first.xyz


In [66]:
a = np.arange(3)
b = np.arange(3) +10

In [73]:
np.stack((a,b))

array([[ 0,  1,  2],
       [10, 11, 12]])